In [1]:
import requests as r
from bs4 import BeautifulSoup
import time
import json
import datetime
import pyodbc
import win32com.client
from selenium import webdriver
import urllib3
import os


In [2]:
mdb_path = r'E:\work\crawlerRA\hkgolden_new.accdb'
headers = {
'Connection': 'keep-alive', 
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
"Accept-Encoding": "gzip, deflate, br", 
"Accept-Language": "zh-CN,zh;q=0.9", 
"origin": "https://forum.hkgolden.com",
"refer": "https://forum.hkgolden.com/",
"Sec-Ch-Ua": "\" Not A;Brand\";v=\"99\", \"Chromium\";v=\"98\", \"Google Chrome\";v=\"98\"", 
"Sec-Ch-Ua-Mobile": "?0", 
"Sec-Ch-Ua-Platform": "\"Windows\"", 
"Sec-Fetch-Dest": "empty", 
"Sec-Fetch-Mode": "cors", 
"Sec-Fetch-Site": "same-site", 
"Sec-Fetch-User": "?1", 
"Upgrade-Insecure-Requests": "1", 
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36", 
"X-Amzn-Trace-Id": "Root=1-63f450b1-0506f77058cb5d0123bb7da7"
}
r.adapters.DEFAULT_RETRIES = 5
s = r.Session()
s.keep_alive = False
s.verify = False
urllib3.disable_warnings()
proxies = {
    'http': 'http://127.0.0.1:7890/',
    'https': 'http://127.0.0.1:7890/'
}

In [28]:
dbname = mdb_path
accApp = win32com.client.Dispatch("Access.Application")
dbEngine = accApp.DBEngine
workspace = dbEngine.Workspaces(0)

dbLangGeneral = ';LANGID=0x0409;CP=1252;COUNTRY=0'
newdb = workspace.CreateDatabase(dbname, dbLangGeneral, 64)
accApp.DoCmd.CloseDatabase
accApp.Quit
newdb = None
workspace = None
dbEngine = None
accApp = None
mdb = u'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+mdb_path
conn = pyodbc.connect(mdb)
cursor = conn.cursor()
cursor.execute("CREATE TABLE Comment_table (Topic_ID int, Comment_ID int, Commenter_ID int, Comment_Text memo, React_Love int, React_Hate int, Reply_To_Floor int, Comment_Date int, Comment_Floor int, Comment_emoji memo)")
cursor.execute("CREATE TABLE Post_table (Class_ID string, Poster_ID int, Topic_ID int, Post_Text memo, Total_Love int, Total_Hate int, Post_Date int)")
cursor.commit()
cursor.close()
conn.close()

In [3]:
def get_all_info(headers, proxies, forum, start_page):
    re = r.get('https://api.hkgolden.com/v1/topics/%d/0?thumb=Y&sort=1&sensormode=Y&filtermodeS=N&hideblock=N&limit=-1'%forum, headers = headers, proxies=proxies)
    maxpage = re.json()['data']['maxPage']
    info_list = []
    topic_id_list = []
    if start_page > maxpage:
        return 'Error'
    for i in range(start_page, maxpage+1):
        info_page = []
        try:
            re = r.get('https://api.hkgolden.com/v1/topics/%d/%d?thumb=Y&sort=1&sensormode=Y&filtermodeS=N&hideblock=N&limit=-1'%(forum, i), headers = headers, proxies=proxies)
        except:
            return 'Error', info_list, i
        else:
            time.sleep(5)
            if re.status_code != 200:
                return 'Error', info_list, i
            if re.json()['result']:
                datas = re.json()['data']['list']
                for data in datas:
                    info_line = []
                    topic_id = data['id']
                    if topic_id in topic_id_list:
                        continue
                    else:
                        topic_id_list.append(topic_id)
                        info_line.append(data['forum'])
                        info_line.append(data['authorId'])
                        info_line.append(topic_id)
                        info_line.append(data['title'])
                        info_line.append(data['marksGood'])
                        info_line.append(data['marksBad'])
                        info_line.append(data['messageDate']//1000)
                        info_page.append(info_line)
            else:
                return 'Error', info_list, i
        info_list+=info_page
    return 'Ok', info_list, i

def fill_post_table(info_list, mdb_path):
    mdb = u'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+mdb_path
    conn = pyodbc.connect(mdb)
    cursor = conn.cursor()
    unique_id_list = []
    for i in info_list:
        topic_id = i[2]
        if topic_id in unique_id_list:
            continue
        else:
            unique_id_list.append(topic_id)
            cursor.execute("INSERT INTO Post_table VALUES('%s', %d, %d, '%s', %d, %d, %d)"%(str(i[0]).replace("'","''"), int(i[1]), int(i[2]), i[3].replace("'","''"), i[4], i[5], i[6]))
    cursor.commit()
    cursor.close()
    conn.close()
    print('Done, total %d datas'%len(info_list))

def fill_comment_table(comment_list, mdb_path):
    mdb = u'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+mdb_path
    conn = pyodbc.connect(mdb)
    cursor = conn.cursor()
    for i in comment_list:
        cursor.execute("INSERT INTO Comment_table VALUES(%d, %d, %d, '%s', %d, %d, %d, %d, %d, '%s')"%(int(i[0]), int(i[1]), int(i[2]), str(i[3]).replace("'","''"), int(i[4]), int(i[5]), int(i[6]), int(i[7]), int(i[8]), str(i[9]).replace("'","''")))
    cursor.commit()
    cursor.close()
    conn.close()
    print('Done, total %d datas'%len(comment_list))
    
def get_comments(headers, proxies, topic_id, start_page):
    while 1:
        re = None
        try:
            re = r.get('https://api.hkgolden.com/v1/view/%d/0?sensormode=Y&hideblock=N'%topic_id, headers = headers, proxies=proxies)
        except:
            print('Error')
            time.sleep(5)
            pass
        if re != None:
            if re.status_code == 200:
                break
    total_page = re.json()['data']['totalPage']
    comment_list = []
    quote_dict = {}
    i = 1
    while 1:
        comment_page = []
        while 1:
            re = None
            try:
                re = r.get('https://api.hkgolden.com/v1/view/%d/%d?sensormode=Y&hideblock=N'%(topic_id, i), headers = headers, proxies=proxies)
            except:
                print('Error')
                time.sleep(5)
                pass
            if re != None:
                if re.status_code == 200:
                    break
        if i == 1:
            comment_list.append([int(topic_id), 0, int(re.json()['data']['authorId']), str(re.json()['data']['content']).replace("'","''"), int(re.json()['data']['marksGood']), int(re.json()['data']['marksBad']), 0, int(re.json()['data']['messageDate'])//1000, 0, ''])
        datas = re.json()['data']['replies']
        if len(datas) == 0:
            break
        i += 1
        for data in datas:
            comment_line = []
            cur_ind = data['index']
            quotes = data['quoted']
            if len(quotes) != 0:
                for quote in quotes:
                    replier_ind = quote['index']
                    quote_dict[replier_ind] = cur_ind
            if cur_ind in list(quote_dict.keys()):
                reply_to = quote_dict[cur_ind]
            else:
                reply_to = 0
            comment_line = [topic_id]
            comment_line.append(data['id'])
            comment_line.append(data['authorId'])
            comment_line.append(data['content'])
            comment_line.append(-1)
            comment_line.append(-1)
            comment_line.append(reply_to)
            comment_line.append(int(data['replyDate'])//1000)
            comment_line.append(int(data['index']))
            comment_line.append('')
            comment_page.append(comment_line)
        comment_list += comment_page
    return 'OK', comment_list, total_page

def get_proxy_from_free_proxy_list(urls=' https://free-proxy-list.net/' ):
        headers = {
            'User-Agent': get_random_user_agent()
        }

        page_text = requests.get(url=urls, headers=headers)
        content = page_text.text
        soup = BeautifulSoup(content, 'lxml')

        table = soup.find_all('table', class_="table table-striped table-bordered")

        df_table = pd.read_html(str(table))[0]
        df_table = df_table[(df_table['Code'] != 'HK') & ((df_table['Google'] == 'yes') | (df_table['Https'] == 'yes'))]
        df_table['proxy'] = df_table.apply(lambda x: [x['IP Address'], x['Port']], axis=1)

In [6]:
re = r.get('https://api.hkgolden.com/v1/view/%d/%d?sensormode=Y&hideblock=N'%(7670808, 1), headers = headers, proxies=proxies)
re.json()

{'result': True,
 'data': {'quoted': [],
  'authorBadge': None,
  'currentPage': 1,
  'content': '<img data-icons=":)" src="/faces/smile.gif" alt=":)" />',
  'tags': [],
  'rated': 0,
  'isVote': False,
  'replies': [],
  'thumbnail': None,
  'authorClass': 1,
  'authorFrameBorder': None,
  'authorIcon': None,
  'isBlocked': False,
  'isBookmarked': False,
  'lastReplies': 0,
  'lastVisit': 0,
  'limitMember': 1,
  'id': 7670808,
  'forum': 'CA',
  'authorId': 487764,
  'authorGender': 1,
  'lastReplyDate': 1678684009320,
  'messageDate': 1678684009320,
  'totalReplies': 0,
  'status': 'A',
  'marksGood': 0,
  'marksBad': 0,
  'title': '楊紫瓊拎奧斯卡印證左中華民族偉大復興',
  'authorName': '老衲',
  'rating': 0,
  'iconType': 'Fresh',
  'iconPath': 'new.gif',
  'totalPage': 1}}

In [46]:
os.environ['NO_PROXY'] = "https://forum.hkgolden.com"
proxies1 = {
    'http': 'http://82.146.48.200:8000/',
    'https': 'http://82.146.48.200:8000/'
}

In [48]:
re = r.get('https://api.hkgolden.com/v1/view/7670323/1?sensormode=Y&hideblock=N', headers = headers, proxies = proxies1, verify=False)

In [49]:
re.json()['data']

{'result': True,
 'data': {'quoted': [],
  'authorBadge': None,
  'currentPage': 1,
  'content': '第一次養貓 希望大家分享下意見<br /><br /><a href="https://cache.hkgolden.media/compress/https://upload.hkgolden.media/comment/qeqffvij.3en3nzpmnrh.of4hee5sq0g.bu5.jpg" target="_blank"><img src="https://cache.hkgolden.media/compress/https://upload.hkgolden.media/comment/qeqffvij.3en3nzpmnrh.of4hee5sq0g.bu5.jpg" alt="https://upload.hkgolden.media/comment/qeqffvij.3en3nzpmnrh.of4hee5sq0g.bu5.jpg" data-src="https://upload.hkgolden.media/comment/qeqffvij.3en3nzpmnrh.of4hee5sq0g.bu5.jpg" /></a><br /><a href="https://cache.hkgolden.media/compress/https://upload.hkgolden.media/comment/vorxt2t1.hram2cxrpt3.4z5ytzfwcuj.lz0.jpg" target="_blank"><img src="https://cache.hkgolden.media/compress/https://upload.hkgolden.media/comment/vorxt2t1.hram2cxrpt3.4z5ytzfwcuj.lz0.jpg" alt="https://upload.hkgolden.media/comment/vorxt2t1.hram2cxrpt3.4z5ytzfwcuj.lz0.jpg" data-src="https://upload.hkgolden.media/comment/vorxt2t1.hram

In [8]:
len(all_topic_ids)

29336

In [10]:
for i in range(1, len(all_topic_ids)):
    [status, comment_list, page] = get_comments(headers, proxies, all_topic_ids[i], 1)
    if status == "OK":
        fill_comment_table(comment_list, mdb_path)
    else:
        print(i)
        break

IndexError: list index out of range

In [5]:
mdb = u'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+mdb_path
conn = pyodbc.connect(mdb)
cursor = conn.cursor()
cursor.execute("SELECT Topic_ID FROM Post_table")
a = cursor.fetchall()
cursor.commit()
cursor.close()
conn.close()

In [6]:
all_topic_ids = [i[0] for i in a]